In [1]:
import sys
import os
root_dir = '/home/qingli/Desktop/Closed-Loop-Learning/nsr/'
os.chdir(root_dir)
from train import *
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme()
sns.set_context("notebook", font_scale=1.5)
sns.color_palette('colorblind')

/home/qingli/.virtualenvs/ans/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[(0.00392156862745098, 0.45098039215686275, 0.6980392156862745),
 (0.8705882352941177, 0.5607843137254902, 0.0196078431372549),
 (0.00784313725490196, 0.6196078431372549, 0.45098039215686275),
 (0.8352941176470589, 0.3686274509803922, 0.0),
 (0.8, 0.47058823529411764, 0.7372549019607844),
 (0.792156862745098, 0.5686274509803921, 0.3803921568627451),
 (0.984313725490196, 0.6862745098039216, 0.8941176470588236),
 (0.5803921568627451, 0.5803921568627451, 0.5803921568627451),
 (0.9254901960784314, 0.8823529411764706, 0.2),
 (0.33725490196078434, 0.7058823529411765, 0.9137254901960784)]

In [2]:
sys.argv = ['']
args = parse_args()
args.dataset = 'hint'
args.split = 'simple'
domain = get_dataset(args.dataset)
args.domain = domain
args.train_set = domain('train', n_sample=args.train_size)
args.val_set = domain('val')
args.test_set = domain('test')

In [24]:
def evaluate(model, dataloader, n_steps=1, log_prefix='val'):
    model.eval() 
    res_all = []
    res_pred_all = []
    
    sent_all = []
    sent_pred_all = []

    dep_all = []
    dep_pred_all = []

    metrics = OrderedDict()

    with torch.no_grad():
        for sample in tqdm(dataloader):
            res = sample['res']
            sent = sample['sentence']
            dep = sample['head']

            res_preds, sent_preds, dep_preds = model.deduce(sample, n_steps=n_steps)
            
            res_pred_all.extend(res_preds)
            res_all.extend(res)
            sent_pred_all.extend(sent_preds)
            sent_all.extend(sent)
            dep_pred_all.extend(dep_preds)
            dep_all.extend(dep)

    pred = res_pred_all
    gt = res_all
    result_acc = np.mean([x == y  for x, y in zip(pred, gt)])
    print("Percentage of missing result: %.2f"%(np.mean([x is MISSING_VALUE for x in pred]) * 100))
    
    pred = [y for x in sent_pred_all for y in x]
    gt = [y for x in sent_all for y in x]
    perception_acc = np.mean([x == y for x,y in zip(pred, gt)])

    SYMBOLS = domain.vocab
    from sklearn.metrics import classification_report, confusion_matrix
    report = classification_report(gt, pred, target_names=SYMBOLS)
    cmtx = confusion_matrix(gt, pred, normalize='all')
    cmtx = pd.DataFrame(
        (10000*cmtx).astype('int'),
        index=SYMBOLS,
        columns=SYMBOLS
    )
    print(report)
    print(cmtx)


    gt_words = gt
    pred = [y for x in dep_pred_all for y in x]
    gt = [y for x in dep_all for y in x]
    head_acc = np.mean(np.array(pred) == np.array(gt))
    head_acc_list = []
    for i, w in enumerate(domain.vocab):
        acc_i = np.mean((np.array(pred) == np.array(gt))[np.array(gt_words) == i])
        print(f"{w}: {acc_i * 100:.2f}")
        head_acc_list.append(acc_i)
    print(f'Digits: {np.mean(head_acc_list[:10])*100:.2f}, Operators: {np.mean(head_acc_list[10:14])*100:.2f}, Parentheses: {np.mean(head_acc_list[14:])*100:.2f}')

    tracked_attrs = getattr(dataloader.dataset, 'tracked_attrs', [])
    for attr in tracked_attrs:
        # print(f"result accuracy by {attr}:")
        attr2ids = getattr(dataloader.dataset, f'{attr}2ids')
        for k, ids in sorted(attr2ids.items()):
            gt = [res_all[i] for i in ids]
            pred = [res_pred_all[i] for i in ids]
            acc = np.mean([x == y  for x, y in zip(pred, gt)]) if ids else 0.
            k = 'div' if k == '/' else k
            metrics[f'result_acc/{attr}/{k}'] = acc

    metrics['result_acc/avg'] = result_acc
    metrics['perception_acc/avg'] = perception_acc
    metrics['head_acc/avg'] = head_acc
    
    print("error cases:")
    errors = [i for i in range(len(res_all)) if res_pred_all[i] != res_all[i]]
    if len(errors) == 0:
        errors = [i for i in range(len(res_all)) if dep_pred_all[i] != dep_all[i]]
    for i in errors[:3]:
        expr = ' '.join([model.config.domain.i2w[x] for x in sent_all[i]])
        expr_pred = ' '.join([model.config.domain.i2w[x] for x in sent_pred_all[i]])
        print(expr)
        print(expr_pred)
        print(dep_all[i])
        print(dep_pred_all[i])
        print(res_all[i])
        print(res_pred_all[i])
        print()
        # tree = draw_parse(expr_pred, dep_pred_all[i])
        # tree.draw()


    return perception_acc, head_acc, result_acc

In [15]:
args.perception = False
args.syntax = True
args.semantics = True
args.resume = 'outputs/wandb/run-20220926_094832-5u4qo99n/ckpt/model_100.p'
model = Jointer(args)
if args.resume:
    st_epoch = model.load(args.resume)

batch_size = 64
eval_dataloader = torch.utils.data.DataLoader(args.test_set, batch_size=batch_size,
                         shuffle=False, num_workers=4, collate_fn=args.domain.collate)
perception_acc, head_acc, result_acc = evaluate(model, eval_dataloader)
print('Iter {}: {} (Perception Acc={:.2f}, Head Acc={:.2f}, Result Acc={:.2f})'.format(0, 'val', 100*perception_acc, 100*head_acc, 100*result_acc))

100%|██████████| 729/729 [1:04:24<00:00,  5.30s/it]


Percentage of missing result: 10.50
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     37211
           1       0.98      0.99      0.98     51360
           2       0.99      1.00      0.99     51769
           3       0.99      1.00      0.99     52691
           4       0.99      1.00      0.99     53107
           5       1.00      1.00      1.00     53310
           6       0.99      0.99      0.99     53805
           7       1.00      0.98      0.99     54277
           8       0.99      1.00      1.00     54434
           9       1.00      0.98      0.99     54716
           +       0.99      1.00      0.99    141105
           -       1.00      1.00      1.00    102765
           *       0.95      0.98      0.96    130088
           /       0.99      0.90      0.94     96102
           (       0.73      1.00      0.84    205661
           )       1.00      0.64      0.78    205661

    accuracy                           0.94 

In [25]:
args.perception = True
args.syntax = False
args.semantics = True
args.resume = 'outputs/wandb/run-20220926_094832-5u4qo99n/ckpt/model_100.p'
model = Jointer(args)
if args.resume:
    st_epoch = model.load(args.resume)

batch_size = 64
eval_dataloader = torch.utils.data.DataLoader(args.test_set, batch_size=batch_size,
                         shuffle=False, num_workers=4, collate_fn=args.domain.collate)
perception_acc, head_acc, result_acc = evaluate(model, eval_dataloader)
print('Iter {}: {} (Perception Acc={:.2f}, Head Acc={:.2f}, Result Acc={:.2f})'.format(0, 'val', 100*perception_acc, 100*head_acc, 100*result_acc))

100%|██████████| 729/729 [01:10<00:00, 10.34it/s]


Percentage of missing result: 0.32
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37211
           1       1.00      1.00      1.00     51360
           2       1.00      1.00      1.00     51769
           3       1.00      1.00      1.00     52691
           4       1.00      1.00      1.00     53107
           5       1.00      1.00      1.00     53310
           6       1.00      1.00      1.00     53805
           7       1.00      1.00      1.00     54277
           8       1.00      1.00      1.00     54434
           9       1.00      1.00      1.00     54716
           +       1.00      1.00      1.00    141105
           -       1.00      1.00      1.00    102765
           *       1.00      1.00      1.00    130088
           /       1.00      1.00      1.00     96102
           (       1.00      1.00      1.00    205661
           )       1.00      1.00      1.00    205661

    accuracy                           1.00  

In [23]:
args.perception = True
args.syntax = True
args.semantics = False
args.resume = 'outputs/wandb/run-20220926_094832-5u4qo99n/ckpt/model_100.p'
model = Jointer(args)
if args.resume:
    st_epoch = model.load(args.resume)

batch_size = 64
eval_dataloader = torch.utils.data.DataLoader(args.test_set, batch_size=batch_size,
                         shuffle=False, num_workers=4, collate_fn=args.domain.collate)
perception_acc, head_acc, result_acc = evaluate(model, eval_dataloader)
print('Iter {}: {} (Perception Acc={:.2f}, Head Acc={:.2f}, Result Acc={:.2f})'.format(0, 'val', 100*perception_acc, 100*head_acc, 100*result_acc))

CUDA is available?: True
using cuda?: True


 17%|█▋        | 127/729 [00:03<00:20, 29.27it/s]

RecursionError()


 19%|█▉        | 141/729 [00:06<01:13,  8.00it/s]

RecursionError()


...

RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()


100%|█████████▉| 726/729 [01:43<00:01,  1.99it/s]

RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()
RecursionError()


100%|██████████| 729/729 [01:43<00:00,  7.05it/s]


RecursionError()
Percentage of missing result: 1.53
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     37211
           1       1.00      1.00      1.00     51360
           2       1.00      1.00      1.00     51769
           3       1.00      1.00      1.00     52691
           4       1.00      1.00      1.00     53107
           5       1.00      1.00      1.00     53310
           6       1.00      1.00      1.00     53805
           7       1.00      1.00      1.00     54277
           8       1.00      1.00      1.00     54434
           9       1.00      1.00      1.00     54716
           +       1.00      1.00      1.00    141105
           -       1.00      1.00      1.00    102765
           *       1.00      1.00      1.00    130088
           /       1.00      1.00      1.00     96102
           (       1.00      1.00      1.00    205661
           )       1.00      1.00      1.00    205661

    accuracy                